In [2]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle


In [10]:
import string
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [32]:
# Load and preprocess data
df = pd.read_csv('C:/Users/HP/Desktop/BE_final/email.csv', encoding='latin-1')
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
df.columns = ["label", "message"]


In [33]:
# Preprocessing function
def preprocess_content(text):
    stemmer = PorterStemmer()
    nopunc = ''.join([char for char in text if char not in string.punctuation])
    words = word_tokenize(nopunc.lower())
    nostop = [stemmer.stem(word) for word in words if word not in stopwords.words('english') and word.isalpha()]
    return ' '.join(nostop)

# Apply preprocessing
df['cleaned_text'] = df['message'].apply(preprocess_content)


In [34]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['cleaned_text'])
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [35]:
# Train model
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Evaluate model
rf_preds = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_preds)
rf_report = classification_report(y_test, rf_preds)

In [36]:
print("Random Forest Model:")
print(f'Accuracy: {rf_accuracy:.4f}')
print(f'Classification Report:\n{rf_report}\n')

Random Forest Model:
Accuracy: 0.4305
Classification Report:
                  precision    recall  f1-score   support

      Interested       0.44      0.41      0.42       494
More information       0.00      0.00      0.00       126
  Not Interested       0.44      0.56      0.49       495

        accuracy                           0.43      1115
       macro avg       0.29      0.32      0.31      1115
    weighted avg       0.39      0.43      0.41      1115




In [37]:
# Prediction function
def predict_class(input_text):
    cleaned_input = preprocess_content(input_text)
    X_new = tfidf.transform([cleaned_input])
    return rf_model.predict(X_new)[0]




In [39]:
# Test prediction
input_text = 'Ok lar... Joking wif u oni...'
predicted_class = predict_class(input_text)
print(f"Predicted class for '{input_text}': {predicted_class}")

Predicted class for 'Ok lar... Joking wif u oni...': Not Interested
